# 410821271 高慎為 電腦視覺HW2

In [39]:
import cv2
import numpy as np

image = cv2.imread('input2.png')

lower_cyan = np.array([90, 200, 100]) 
upper_cyan = np.array([255, 255, 130])
lower_red = np.array([0, 0, 150]) 
upper_red = np.array([25, 25, 255])

cyan_mask = cv2.inRange(image, lower_cyan, upper_cyan)
red_mask = cv2.inRange(image, lower_red, upper_red)

cv2.imshow('Red Object', red_mask)
cv2.imshow('Cyan Object', cyan_mask)
cv2.imshow('Input', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [40]:
import cv2
import numpy as np

ret, binary_cyan = cv2.threshold(cyan_mask,127,255,cv2.THRESH_BINARY)
contours_cyan, hierarchy = cv2.findContours(binary_cyan,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
cnt_cyan = contours_cyan[0]
epsilon_cyan = 0.01 * cv2.arcLength(cnt_cyan, True)
approx_cyan = cv2.approxPolyDP(cnt_cyan, epsilon_cyan, True)
img_cyan = cv2.cvtColor(cyan_mask, cv2.COLOR_GRAY2BGR)
cv2.polylines(img_cyan, [approx_cyan], True, (0, 0, 255), 2)

#找中心點
center = cv2.moments(cnt_cyan)
cX = int(center["m10"] / center["m00"])
cY = int(center["m01"] / center["m00"])
c_cyan = (cX,cY)
#print(c_cyan)

#計算輪廓的每個點到中心點的距離
P = []
for point in approx_cyan:
    distance_to_center = np.sqrt((point[0][0] - cX)**2 + (point[0][1] - cY)**2)
    P.append(distance_to_center)

#找出離中心點最遠的三個點
far_index = np.argsort(P)[-3:]
far_point = [approx_cyan[i] for i in far_index]
far_point = [point[0].tolist() for point in far_point]
#print(far_point)

#找出離中心點最近的那個點
near_index = np.argmin(P)
near_point = approx_cyan[near_index]
near_point = near_point[0].tolist()
#print(near_point)
#cv2.circle(img_cyan, tuple(near_point), 2, (0, 255, 0), -1)
#for point in far_point:
    #cv2.circle(img_cyan, tuple(point), 2, (0, 255, 0), -1)

#用離中心最近的點去判斷出箭頭的方向    
near_point_np = np.array(near_point)
far_point_np = np.array(far_point)
distances = np.linalg.norm(far_point_np - near_point_np, axis=1)
farthest_index = np.argmax(distances)
farthest_point = far_point[farthest_index]     
#cv2.circle(img_cyan, tuple(farthest_point), 2, (0, 255, 0), -1)

#畫一條黃線指示玩家的方向
vector = np.array(farthest_point) - np.array(c_cyan)
normalized_vector = vector / np.linalg.norm(vector)
end_point = tuple(np.array(c_cyan) + 20 * normalized_vector)
cv2.line(image, c_cyan, tuple(map(int, end_point)), (0, 255, 255), 2)

#cv2.imshow("img cyan", img_cyan)
cv2.imshow("img", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
import cv2
import numpy as np

ret, binary_red = cv2.threshold(red_mask,127,255,cv2.THRESH_BINARY)
contours_red, hierarchy = cv2.findContours(binary_red,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

cnt_red = contours_red[4]
epsilon_red = 0.01 * cv2.arcLength(cnt_red, True)
approx_red = cv2.approxPolyDP(cnt_red, epsilon_red, True)
img_red = cv2.cvtColor(red_mask, cv2.COLOR_GRAY2BGR)
cv2.polylines(img_red, [approx_red], True, (0, 0, 255), 2)

center = cv2.moments(cnt_red)
cX = int(center["m10"] / center["m00"])
cY = int(center["m01"] / center["m00"])
c_red = (cX,cY)
#cv2.circle(img_red, tuple(c_red), 2, (0, 255, 0), -1)

cv2.line(image, c_cyan, c_red, (0, 255, 255), 2)

#cv2.imshow("img red", img_red)
cv2.imshow("img", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
import numpy as np

farthest_point = tuple(farthest_point)

#print(farthest_point)
#print(c_cyan)
#print(c_red)

vector_1 = np.array(farthest_point) - np.array(c_cyan)
vector_2 = np.array(c_red) - np.array(c_cyan)

#print(vector_1)
#print(vector_2)

dot = np.dot(vector_1, vector_2)
norm_1 = np.linalg.norm(vector_1)
norm_2 = np.linalg.norm(vector_2)

angle_rad = np.arccos(dot / (norm_1 * norm_2))
angle_deg = np.degrees(angle_rad)

cross_product = np.cross(vector_1, vector_2)
if cross_product > 0:
    print("+", angle_deg)
elif cross_product < 0:
    print("-", angle_deg)




- 45.62959860841027
